## 브런치 카테고리별 게시글 크롤러

> <b>1.</b> 카테고리별 게시글 목록 화면에서 해당글의 url을 parsing <br>
> <b>2.</b> 게시글별 url을 request 후 필요한 부분 parsing <br>
> <b>3.</b> 추출한 데이터를 json형식으로 변환 후 파일저장(.json)

In [1]:
# !pip install selenium

In [1]:
# module import

import urllib
import json
from bs4 import BeautifulSoup
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import NoAlertPresentException

from selenium.webdriver.chrome.options import Options

import seaborn as sns
import sys
import time
import networkx as nx
import matplotlib.pyplot as plt
import re
from collections import defaultdict
%matplotlib inline

> 카테고리별 게시글목록 페이지의 경우 무한스크롤 이벤트가 걸려있다. 따라서 selenium을 사용하여 스크롤이 한번 끝나는 시점에 자동으로 다음 스크롤 이벤트가 시행되도록 하였다.

In [4]:
# selenium 속도향상위해 불필요한 옵션을 사용하지 않게하는 코드

options = Options()
prefs = {'profile.default_content_setting_values': {'cookies' : 2, 'images': 2, 
                                                    'plugins' : 2, 'popups': 2, 'geolocation': 2,
                                                    'notifications' : 2, 'auto_select_certificate': 2,
                                                    'fullscreen' : 2,
                                                    'mouselock' : 2, 'mixed_script': 2, 
                                                    'media_stream' : 2, 'media_stream_mic' : 2,
                                                    'media_stream_camera': 2, 'protocol_handlers' : 2,
                                                    'ppapi_broker' : 2, 'automatic_downloads': 2, 'midi_sysex' : 2,
                                                    'push_messaging' : 2, 'ssl_cert_decisions': 2, 'metro_switch_to_desktop' : 2,
                                                    'protected_media_identifier': 2, 'app_banner': 2, 'site_engagement' : 2,
                                                    'durable_storage' : 2}}

options.add_experimental_option('prefs', prefs) 
options.add_argument("start-maximized") 
options.add_argument("disable-infobars") 
options.add_argument("--disable-extensions")

> 게시글 목록에서 게시글별 url을 파싱하는 부분

In [5]:
# 전체 카테고리 목록
# category_list = ['지구한바퀴_세계여행?q=g','그림·웹툰?q=g','시사·이슈?q=g','IT_트렌드?q=g','사진·촬영?q=g','',
#                  '취향저격_영화_리뷰?q=g','오늘은_이런_책?q=g','뮤직_인사이드?q=g','글쓰기_코치?q=g','직장인_현실_조언?q=g'
#                  ,'스타트업_경험담?q=g','육아_이야기?q=g','요리·레시피?q=g','건강·운동?q=g','멘탈_관리_심리_탐구?q=g',
#                  '디자인_스토리?q=g','문화·예술?q=g','건축·설계?q=g','인문학·철학?q=g','쉽게_읽는_역사?q=g','우리집_반려동물?q=g',
#                  '멋진_캘리그래피?q=g','사랑·이별?q=g','감성_에세이?q=g']

category_list = ['직장인_현실_조언?q=g'] # input category
idlist = []

# 스크롤 시 마다 url을 파싱해오는 함수 정의
def crawlBrunchLink(dir):
    ## html crawling
    chromedriver = 'C:/selenium/chromedriver.exe' 
    driver = webdriver.Chrome(chromedriver) # 크롬드라이버 작동
    base_url = "https://brunch.co.kr/keyword/{dir}".format(dir=dir) # 시작 url
    verificationErrors = []
    accept_next_alert = True
#     delay = 3
    driver.get(base_url) # chrome driver에 시작 url 얹음
    #driver.find_element_by_link_text("All").click()

    cnt = 0
    while True : # 스크롤 횟수
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") # 페이지 무한 스크롤
        driver.implicitly_wait(10)
        time.sleep(5)
        cnt = cnt + 1
        if cnt % 20 == 0: # 스크롤 횟수20번마다 날짜 체크
            html_source = driver.page_source
            data = html_source.encode('utf-8')
            soup = BeautifulSoup(data, 'html.parser')
            pd = soup.find_all('span',{'class':'publish_time'})
            if (len(re.findall('Sep [\d]{1,2}\. 2018', pd[len(pd)-1].text)) != 0) or (len(re.findall('Aug [\d]{1,2}\. 2018', pd[len(pd)-1].text)) != 0) or (len(re.findall('Jul [\d]{1,2}\. 2018', pd[len(pd)-1].text)) != 0) or (len(re.findall('Jun [\d]{1,2}\. 2018', pd[len(pd)-1].text)) != 0):     
                break
            else : 
                pass

    html_source = driver.page_source
    classes = soup.find_all('li',{'class':'list_has_image animation_up'})

    for c in classes:
        follwing = c.get('data-articleuid')
        idlist.append(follwing.replace('_','/')) # 
    
#     driver.quit()
    driver.close()
    return idlist

In [6]:
writer_list = [] # 크롤링 해온 해당글 url 리스트

for dir in category_list:
    idlist =[] # idlist 누적방지
    crawlBrunchLink(dir)
    writer_list.append(idlist)

KeyboardInterrupt: 

> 크롤링 중간에 오류가 발생할경우를 대비하여 중간중간 pickle 파일로 저장

In [120]:
import pickle

# ## Save pickle
# with open("writer_list_pickle.pickle","wb") as fw:
#     pickle.dump(my_list, fw)
 
## Load pickle
# with open("writer_list_그림웹툰_pickle.pickle","rb") as fr:
#     writer_list_import = pickle.load(fr)

# writer_list = []
# writer_list.append(writer_list_import)

In [123]:
len(writer_list[0])

15052

> 게시글의 url을 모두 가져온 후, beautifulsoup을 이용해 해당 url에서 필요한 정보들을 parsing한다

In [113]:
## 게시글 크롤링

import requests
import json

data = []
json_data = {}
# res_text = []

def def_craw(writer_list,input_catenum):
    global json_data
    global data
    res_text = []
    tag_keyword=[]
    
    tag_title,tag_nickname,tag_publish_date,tag_url,tag_url_plink = None,None,None,None,None
    tag_share,tag_like = str,str
    for i,url in enumerate(writer_list[input_catenum]):
        print("전체%d개의 게시글 중 %d번째 글 진행중" %(len(writer_list[input_catenum]),i+1))
        if res_text == []: # 첫 시작에러 방지
            pass
        else :
            
            # to json
            json_data['title'] = tag_title  
            json_data['nickname'] = tag_nickname
            json_data['publish_date'] = tag_publish_date
            json_data['keyword'] = tmp_keyword   
            json_data['like'] = tag_like # like 없는 경우 ''
            json_data['share'] = tag_share # share 없는 경우 None
            json_data['comment'] = tag_comment # comment 없는 경우 ''
            json_data['url'] = tag_url
            json_data['url_plink'] = tag_url_plink 
            json_data['text'] = res_text

        data.append(json_data)
#         print(data)
        
        json_data = {} # 누적방지 초기화
        tmp_keyword = [] # 누적방지 초기화
        res_text = [] # 누적방지 초기화 
        
        # beautifulsoup
        html = requests.get('https://brunch.co.kr/@@{text_url}'.format(text_url=url))
        soup = BeautifulSoup(html.text, 'html.parser')
        
        if soup.find('title').text == "brunch":
            pass
        else:
            tag_title = soup.find('title').text # 해당페이지의 Title
            tag_url = soup.find("meta",property='og:url')['content'] # 본주소
            tag_nickname = soup.find("meta",{'name':'article:media_name'})['content'] # 글쓴이 닉네임
            tag_url_plink = soup.find("meta",property='dg:plink')['content'] # 암호주소? # 모바일?
            tag_publish_date = soup.find("meta",property='article:published_time')['content'] # 본주소
            tag_keyword = soup.find_all('a',href=re.compile('/keyword')) # 해당 글의 키워드
            tag_like = soup.find('span',{'class':'f_l text_like_count text_default text_with_img_ico ico_likeit_like #like'}) #좋아요 수
            tag_share = soup.find('span',{'class':'f_l text_share_count text_default text_with_img_ico'}) # 공유 수
            tag_comment = soup.find('span',{'class':'f_l text_comment_count text_default text_with_img_ico'}) # 댓글 수
            tag_url_meta = soup.find("meta",property='ks:richscrap')['content']
        
            to_json = json.loads(tag_url_meta)
            for i in to_json['cards']:
                if i['type'] == 'text':
                    res_text.append(i['body']['content'])

            if tag_like == None:
                tag_like = "0"
            else:
                tag_like = tag_like.text # 좋아요 수

            if tag_share == None:
                tag_share == "0"
            else:
                tag_share = tag_share.text # 공유 수

            if tag_comment == None:
                tag_comment =="0"
            else:
                tag_comment = tag_comment.text

            for keyword in tag_keyword:
                tmp_keyword.append(keyword.text)

In [125]:
# parsing 실행
# def_craw(writer_list, "실행시킬 카테고리 Num")
def_craw(writer_list,0)

전체15052개의 게시글 중 1번째 글 진행중
전체15052개의 게시글 중 2번째 글 진행중
전체15052개의 게시글 중 3번째 글 진행중
전체15052개의 게시글 중 4번째 글 진행중
전체15052개의 게시글 중 5번째 글 진행중
전체15052개의 게시글 중 6번째 글 진행중
전체15052개의 게시글 중 7번째 글 진행중
전체15052개의 게시글 중 8번째 글 진행중
전체15052개의 게시글 중 9번째 글 진행중
전체15052개의 게시글 중 10번째 글 진행중
전체15052개의 게시글 중 11번째 글 진행중
전체15052개의 게시글 중 12번째 글 진행중
전체15052개의 게시글 중 13번째 글 진행중
전체15052개의 게시글 중 14번째 글 진행중
전체15052개의 게시글 중 15번째 글 진행중
전체15052개의 게시글 중 16번째 글 진행중
전체15052개의 게시글 중 17번째 글 진행중
전체15052개의 게시글 중 18번째 글 진행중
전체15052개의 게시글 중 19번째 글 진행중
전체15052개의 게시글 중 20번째 글 진행중
전체15052개의 게시글 중 21번째 글 진행중
전체15052개의 게시글 중 22번째 글 진행중
전체15052개의 게시글 중 23번째 글 진행중
전체15052개의 게시글 중 24번째 글 진행중
전체15052개의 게시글 중 25번째 글 진행중
전체15052개의 게시글 중 26번째 글 진행중
전체15052개의 게시글 중 27번째 글 진행중
전체15052개의 게시글 중 28번째 글 진행중
전체15052개의 게시글 중 29번째 글 진행중
전체15052개의 게시글 중 30번째 글 진행중
전체15052개의 게시글 중 31번째 글 진행중
전체15052개의 게시글 중 32번째 글 진행중
전체15052개의 게시글 중 33번째 글 진행중
전체15052개의 게시글 중 34번째 글 진행중
전체15052개의 게시글 중 35번째 글 진행중
전체15052개의 게시글 중 36번째 글 진행중
전체15052개의 게시글 중 37번째 글 진행중
전체15052개의 

In [126]:
len(data)

15052

> title,작가ID,작성일 등등의 데이터가 json 형식으로 저장된것을 확인

In [127]:
data

[{},
 {},
 {'title': '[레소토 여행] 기념품 사는 건 어려워',
  'nickname': '대린나',
  'publish_date': '2019-10-25T05:13+0900',
  'keyword': ['기념품', '아프리카여행', '웹툰'],
  'like': '1',
  'share': None,
  'comment': '',
  'url': 'https://brunch.co.kr/@derinna/151',
  'url_plink': 'https://brunch.co.kr/@@5T3o/151',
  'text': ['         ',
   ' ',
   ' ',
   ' ',
   '마세루 도심의 기념품 가게 바수투햇(Basotho Hat, 레소토 모자라는 뜻). 기념품 가게답게 레소토 전통 모자를 본딴 모양으로 지어졌다. 전통 그릇, 짚 모자(모코로초), 조각품, 가방, 인형 등 엄청나게 많은 기념품이 있었다. ',
   ' ',
   ' ',
   '이곳에서 36랜드(=36말로티)에 팔리고 있던 수투 여인 인형은 내가 사니패스 여행에서 150랜드에 산 것과 똑같았다. 사니패스의 아주머니가 직접 만든 인형이라고 안 깎아주려던 것을 흥정해서 50랜드 싸게 산 것이었다. 그런데 왠걸. 사니패스에서 만들어졌다는 인형이 왜 여기에 훨씬 많은 것이며, 또 왜 가격은 5배나 싼 것인가... 사람 좋은 얼굴을 하고 그렇게나 바가지를 씌우다니.. 이미 몇 달이나 지난 일이지만 당했다는 생각에 기분이 나빠졌다.']},
 {'title': '마흔이 넘어 찾아 온 일상예술',
  'nickname': '이선영',
  'publish_date': '2019-10-25T03:10+0900',
  'keyword': ['그림일기', '드로잉', '글쓰기'],
  'like': '2',
  'share': None,
  'comment': '',
  'url': 'https://brunch.co.kr/@camin2002/12',
  'url_plink': '

> 추출한 데이터를 json 파일로 저장한다

In [128]:
import json
from collections import OrderedDict

# Ready for data
# OrderDict => 딕셔너리 key 자동정렬 방지
del data[0] # 위 코드상으로 data 맨앞에 빈 dict들어감으로 여기서 빈 dict 삭제
to_json = OrderedDict() 
full_data = OrderedDict()

to_json['name'] = "그림웹툰" # 카테고리명으로 지정 
to_json['version'] = "2019-10-25" # 날짜
to_json['data'] = data

# Write JSON
# json 저장 부분. 이름 바꿔서 저장해야함
# brunch_json_category_날짜+버전 형식으로 저장할것
with open('bruch_json_그림웹툰_1910251', 'w', encoding="utf-8") as make_file: 
    json.dump(to_json, make_file, ensure_ascii=False, indent="\t")
    
data = []

In [60]:
# html = requests.get('https://brunch.co.kr/@{text_url}'.format(text_url='@3799/35'))
# soup = BeautifulSoup(html.text, 'html.parser')
# tag_title = soup.find('title') # 해당페이지의 Title
# tag_nickname = soup.find("meta",{'name':'article:media_name'})['content'] # 글쓴이 닉네임
# tag_url = soup.find("meta",property='og:url')['content'] # 본주소
# tag_url_plink = soup.find("meta",property='dg:plink')['content'] # 암호주소? # 모바일?
# tag_publish_date = soup.find("meta",property='article:published_time')['content'] # 본주소
# tag_keyword = soup.find_all('a',href=re.compile('/keyword')) # 해당 글의 키워드
# tag_like = soup.find('span',{'class':'f_l text_like_count text_default text_with_img_ico ico_likeit_like #like'}) # 좋아요 수
# tag_share = soup.find('span',{'class':'f_l text_share_count text_default text_with_img_ico'}) # 공유 수
# tag_comment = soup.find('span',{'class':'f_l text_comment_count text_default text_with_img_ico'}) # 댓글 수는 이미지임

# tag_url_meta = soup.find("meta",property='ks:richscrap')['content']
